# 한빛아카데미 
https://www.hanbit.co.kr/academy/books/new_book_list.html?page=1

In [ ]:
# !pip install tqdm  진행사항을 100% bar형태로 알려줌

In [4]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
from tqdm import tqdm


In [5]:
base_url = 'https://www.hanbit.co.kr'
main_url = '/academy/books/new_book_list.html'
url = f'{base_url}{main_url}?page=1'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [8]:
lis = soup.select('.sub_book_list')
len(lis)

20

In [ ]:
li = lis[10]
li

In [18]:
sub_href = li.select_one('.book_tit > a')['href']
sub_href

'/academy/books/book_view.html?p_code=B9678437685'

In [19]:
# 서브페이지에서 정보를 추출
sub_url = base_url + sub_href
sub_res = requests.get(sub_url)
sub_soup = BeautifulSoup(sub_res.text, 'html.parser')

In [20]:
title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
title


'STEM CookBook, TAO 해석학 Ⅱ(4판)'

In [21]:
# 책 정보의 딕셔너리 생성
sub_lis = sub_soup.select('.info_list > li')
info_dict = {}
for sub_li in sub_lis[:-1]:
    key = sub_li.find('strong').get_text().strip()
    value = sub_li.find('span').get_text().strip()
    info_dict[key[:-2]] = value
info_dict

{'저자': 'Terence Tao',
 '번역': '한빛수학교재연구소 , 권순식(감수)',
 '출간': '2023-10-16',
 '페이지': '252 쪽',
 'ISBN': '9791156646808',
 '물류코드': '4680'}

In [22]:
price = int(sub_soup.select_one('.pbr > strong').string.replace(',', ''))
price

21000

In [24]:
data = []
for li in lis:
    sub_href = li.select_one('.book_tit > a')['href']
    sub_url = base_url + sub_href
    sub_res = requests.get(sub_url)
    sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
    title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
    sub_lis = sub_soup.select('.info_list > li')
    info_dict = {}
    for sub_li in sub_lis[:-1]:
        key = sub_li.find('strong').get_text().strip()
        value = sub_li.find('span').get_text().strip()
        info_dict[key[:-2]] = value
    price = int(sub_soup.select_one('.pbr > strong').string.replace(',', ''))

    #info_dict에서 저자 역자 출간일 페이지 추출
    author = ', '.join(info_dict['저자'].split(' , '))
    translator = info_dict['번역'].replace(' ,', ',') if '번역' in info_dict.keys() else ' '
    date = info_dict['출간']
    page = int(info_dict['페이지'][:-2])

    data.append({'제목' : title, '저자' : author, '역자':translator, '출간일':date,
                 '페이지수': page, '가격':price})

In [30]:
df = pd.DataFrame(data)

In [26]:
df.head()

,제목,저자,역자,출간일,페이지수,가격
0,"IT@CookBook, 예제로 배우는 회로이론",김명식,,2023-12-10,696,32000
1,4차 산업혁명 시대의 창업실무(2판),"윤두열, 이정이",,2023-12-08,436,29000
2,"IT CookBook, 쉽게 배우는 JSP 웹 프로그래밍(2판)",송미영,,2023-12-08,676,30000
3,"IT CookBook, 핵심이 보이는 신호 및 시스템(2판)",이철희,,2023-12-08,688,33000
4,"STEM CookBook, 수학의 깊이를 더하는 『STEIN 복소해석학』 강의노트",김영원,,2023-11-20,200,22000


In [27]:
# 전체 페이지에서 정보를 추출하기
import requests
import pandas as pd 
from bs4 import BeautifulSoup
from tqdm import tqdm
base_url = 'https://www.hanbit.co.kr'
main_url = '/academy/books/new_book_list.html'

data1 = []
for page in tqdm(range(1, 25)):
    url = f'{base_url}{main_url}?page={page}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    for li in lis:
        sub_href = li.select_one('.book_tit > a')['href']
        sub_url = base_url + sub_href
        sub_res = requests.get(sub_url)
        sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
        title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
        sub_lis = sub_soup.select('.info_list > li')
        info_dict = {}
        for sub_li in sub_lis[:-1]:
            key = sub_li.find('strong').get_text().strip()
            value = sub_li.find('span').get_text().strip()
            info_dict[key[:-2]] = value
        price = int(sub_soup.select_one('.pbr > strong').string.replace(',', ''))

        #info_dict에서 저자 역자 출간일 페이지 추출
        author = ', '.join(info_dict['저자'].split(' , '))
        translator = info_dict['번역'].replace(' ,', ',') if '번역' in info_dict.keys() else ' '
        date = info_dict['출간']
        page = int(info_dict['페이지'][:-2])

        data1.append({'제목' : title, '저자' : author, '역자':translator, '출간일':date,
                    '페이지수': page, '가격':price})   

100%|██████████| 24/24 [02:12<00:00,  5.52s/it]


In [28]:
df = pd.DataFrame(data1)
df.tail()

,제목,저자,역자,출간일,페이지수,가격
475,패널데이터와 복합표본을 활용한 한번에 적용하는 의학보건통계,히든그레이스 데이터분석팀,,2023-08-18,468,34000
476,"IT CookBook, HTML5 웹 프로그래밍 입문(4판)",윤인성,,2023-08-05,500,29000
477,"STEM CookBook, 경영경제수학 첫걸음","김진한, 이재원",,2023-07-31,476,26000
478,논리력을 키우는 영어 글쓰기,이요안,,2023-07-28,248,20000
479,"IT CookBook, 난생처음 데이터 분석 with 파이썬","김규석, 김주희, 이준",,2023-07-24,512,26000


In [29]:
df.to_csv('data/한빛신간도서.csv', index=False)